In [1]:
import torch

In [5]:
from transformers import AutoTokenizer, OPTForCausalLM, BloomForCausalLM, CodeGenForCausalLM

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import transformers
transformers.__version__

'4.33.1'

In [7]:
model = OPTForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

In [6]:
# b_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
# b_model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

In [7]:
# c_tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
# c_model = CodeGenForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

In [59]:
def pre_hook(module, args, kwargs):
    # print(args, kwargs)
    display = {}
    for k, v in kwargs.items():
        if isinstance(v, torch.Tensor):
            display[k] = v.shape 
        elif k == 'past_key_values' and v is not None:
            display[k] = f'{len(v)} * ' + str((v[0][0].shape, v[0][1].shape))
        else:
            display[k] = v
    print(display)

    return args, kwargs

model._forward_pre_hooks.clear()
model.register_forward_pre_hook(pre_hook, with_kwargs=True)
model._forward_pre_hooks



OrderedDict([(26, <function __main__.pre_hook(module, args, kwargs)>)])

In [62]:
prompts = [
    'Who are you? Are you conscious?',
    'Where is Deutschland?',
    'How is Huawei Mate 60 Pro?'
] * 4

prompt_len = 20

inputs = tokenizer(prompts, padding="max_length", max_length=prompt_len, return_tensors="pt")

# Generate
generate_ids = model.generate(
    inputs.input_ids, 
    max_length=30 + prompt_len,
    num_beams=2,
    # num_beam_groups=2,
    # diversity_penalty=0.1,
    # do_sample=True,
)

output_texts = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
for output_text in output_texts:
    print(output_text)
    print('-' * 10)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


{'input_ids': torch.Size([24, 20]), 'past_key_values': None, 'use_cache': True, 'attention_mask': torch.Size([24, 20]), 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': torch.Size([24, 1]), 'past_key_values': '12 * (torch.Size([24, 12, 20, 64]), torch.Size([24, 12, 20, 64]))', 'use_cache': True, 'attention_mask': torch.Size([24, 21]), 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': torch.Size([24, 1]), 'past_key_values': '12 * (torch.Size([24, 12, 21, 64]), torch.Size([24, 12, 21, 64]))', 'use_cache': True, 'attention_mask': torch.Size([24, 22]), 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': torch.Size([24, 1]), 'past_key_values': '12 * (torch.Size([24, 12, 22, 64]), torch.Size([24, 12, 22, 64]))', 'use_cache': True, 'attention_mask': torch.Size([24, 23]), 'return_dict': True, 'output_attentions': False, 'output_hidden_states': False}
{'input_ids': 